In [90]:
pip install transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [91]:
pip install vncorenlp

Note: you may need to restart the kernel to use updated packages.


In [92]:
!mkdir -p vncorenlp/models/wordsegmenter

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar

!mv VnCoreNLP-1.1.1.jar vncorenlp/



!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr

!mv vi-vocab vncorenlp/models/wordsegmenter/

!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/





!mkdir -p vncorenlp/models/postagger

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/postagger/vi-tagger

!mv vi-tagger vncorenlp/models/postagger/





!mkdir -p vncorenlp/models/ner

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-500brownclusters.xz

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-ner.xz

!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/ner/vi-pretrainedembeddings.xz

!mv vi-500brownclusters.xz vncorenlp/models/ner/

!mv vi-ner.xz vncorenlp/models/ner/

!mv vi-pretrainedembeddings.xz vncorenlp/models/ner/

--2024-12-11 10:37:42--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: 'VnCoreNLP-1.1.1.jar'

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.09s   

2024-12-11 10:37:42 (285 MB/s) - 'VnCoreNLP-1.1.1.jar' saved [27412575/27412575]

--2024-12-11 10:37:44--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respon

In [93]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [94]:
pip install keras-preprocessing


Note: you may need to restart the kernel to use updated packages.


In [95]:
VNCORE_NLP = False

max_len = 50

In [96]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

In [97]:
def tokennize_vn(text):

    sentences = vncorenlp.tokenize(text)

    s = ''

    for t in sentences:

        s = s + ' '.join(t) + ' '

    return s

In [98]:
train_dataset = '/kaggle/input/vithsd-norm/Train_normalize.csv'

dev_dataset = '/kaggle/input/vithsd-aug-norm/Dev_normalize.csv'

test_dataset = '/kaggle/input/vithsd-aug-norm/Test_normalize.csv'

In [99]:
import pandas as pd



train = pd.read_csv(train_dataset)

dev = pd.read_csv(dev_dataset)

test = pd.read_csv(test_dataset)

In [100]:
import itertools



aspect = list(train.columns.values[2:])

sent = [1,2,3]



itertools.product([aspect, sent])



combined_label = [a for a in itertools.product(aspect, sent)]

In [101]:
import numpy as np

import pandas as pd



def make_data(data):

    content = []

    label = []



    for i in range(0, len(data)):

        row = data.iloc[i]

        if VNCORE_NLP:

            content.append(tokennize_vn(str(row['content'])))

        else:

            content.append(row['content'])

        default_lab = np.zeros(len(combined_label), dtype=int)

        for j in range(0, len(combined_label)):

            l = combined_label[j]

            if row[l[0]] == l[1]:

                default_lab[j] = 1

        label.append(default_lab)



    return pd.DataFrame({

        'Content': content,

        'Labels': label

    })

In [102]:
train_data = make_data(train)

dev_data = make_data(dev)

test_data = make_data(test)

In [103]:
train_data['Content'] = train_data['Content'].astype(str)
dev_data['Content'] = dev_data['Content'].astype(str)
test_data['Content'] = test_data['Content'].astype(str)


In [104]:
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.preprocessing.text import Tokenizer



tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_data['Content'])

vocab_size = len(tokenizer.word_index) + 1



def encoding(X, y):

    X = tokenizer.texts_to_sequences(X.values.astype(str))

    X = pad_sequences(X, maxlen=max_len, padding='post')

    y = np.asarray([np.asarray(row, dtype=float) for row in y], dtype=int)

    return (X,y)

In [105]:
train_features = encoding(train_data['Content'], train_data['Labels'])

dev_features = encoding(dev_data['Content'], dev_data['Labels'])

test_features = encoding(test_data['Content'], test_data['Labels'])

In [106]:
# Evaluation metric

import sys

import os

import os.path

from scipy.stats import sem

import numpy as np

from ast import literal_eval

import tensorflow as tf



def em_score(y_true, y_pred):

    MR = np.all(y_pred == y_true, axis=1).mean()

    return MR



def accuracy_score(y_true, y_pred):

    temp = 0

    for i in range(0, len(y_true)):

        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    
    return temp / len(y_true)


def f1_score(y_true, y_pred):

    temp = 0

    for i in range(len(y_true)):

        if (sum(y_true[i]) == 0) and (sum(y_pred[i]) == 0):

            continue

        temp+= (2*sum(np.logical_and(y_true[i], y_pred[i]))) / (sum(y_true[i])+sum(y_pred[i]))

    return temp/ len(y_true)

def recall(y_true, y_pred):
    temp = 0

    for i in range(len(y_pred)):
        if(sum(y_true[i]) == 0) and (sum(y_pred[i])==0):
            continue
        if sum(y_true[i]) == 0:
            continue
        temp += sum(np.logical_and(y_true[i],y_pred[i])) / sum(y_true[i])
    return temp / len(y_pred)

def precision(y_true, y_pred):
    temp = 0

    for i in range(len(y_pred)):
        if(sum(y_true[i]) == 0) and (sum(y_pred[i])==0):
            continue
        if sum(y_pred[i]) ==0:
            continue
        temp += sum(np.logical_and(y_true[i],y_pred[i])) / sum(y_pred[i])
    return temp / len(y_pred)

In [107]:
from transformers import AutoModel

#Thêm model mà yêu cầu
model = AutoModel.from_pretrained('uitnlp/visobert')
embedding_matrix = model.embeddings.word_embeddings.weight.detach().numpy()
dim = embedding_matrix.shape[1]
vocab = embedding_matrix.shape[0]


config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/390M [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at uitnlp/visobert and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [108]:
import tensorflow as tf

from tensorflow import keras



from keras.models import Sequential, Model



from keras.layers import Dense, Concatenate, Bidirectional, GRU, Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D, SpatialDropout1D, Dropout, concatenate

from keras.layers import Flatten, LSTM, Input

from tensorflow.keras.layers import Embedding



units = 100


input = Input(shape = (max_len,))

emb = Embedding(vocab, dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(input)

x1 = SpatialDropout1D(0.2)(emb)



x = Bidirectional(GRU(units, return_sequences = True))(x1)

x = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)



y = Bidirectional(LSTM(units, return_sequences = True))(x1)

y = Conv1D(int(units/2), kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(y)



avg_pool1 = GlobalAveragePooling1D()(x)

max_pool1 = GlobalMaxPooling1D()(x)



avg_pool2 = GlobalAveragePooling1D()(y)

max_pool2 = GlobalMaxPooling1D()(y)





x = Concatenate(axis=-1)([avg_pool1, max_pool1, avg_pool2, max_pool2])

out = Dense(15, activation = "sigmoid")(x)



model = Model(input, out)



model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 50, 768)   │ 11,523,072 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_3 │ (None, 50, 768)   │          0 │ embedding_3[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_6     │ (None, 50, 200)   │    522,000 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_7     │ (None, 50, 200)   │    695,200 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 49, 50)    │     20,050 │ bidirectional_6[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 49, 50)    │     20,050 │ bidirectional_7[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ conv1d_6[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 50)        │          0 │ conv1d_6[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 50)        │          0 │ conv1d_7[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 50)        │          0 │ conv1d_7[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 200)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_average_p… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 15)        │      3,015 │ concatenate_3[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 12,783,387 (48.76 MB)

 Trainable params: 12,783,387 (48.76 MB)

 Non-trainable params: 0 (0.00 B)

In [109]:
model.fit(train_features[0].astype(np.float32), train_features[1], validation_data=(dev_features[0].astype(np.float32), dev_features[1]),

          batch_size=128, epochs=20, verbose=True)

Epoch 1/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - acc: 0.2944 - loss: 0.3176 - val_acc: 0.5612 - val_loss: 0.2163
Epoch 2/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.5790 - loss: 0.2101 - val_acc: 0.6062 - val_loss: 0.1913
Epoch 3/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.6559 - loss: 0.1765 - val_acc: 0.6087 - val_loss: 0.1843
Epoch 4/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.6982 - loss: 0.1480 - val_acc: 0.5595 - val_loss: 0.1965
Epoch 5/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.7148 - loss: 0.1254 - val_acc: 0.5670 - val_loss: 0.2042
Epoch 6/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.7369 - loss: 0.1015 - val_acc: 0.5454 - val_loss: 0.2253
Epoch 7/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - acc: 0.7601 - loss: 0.0856 - val_acc: 0.5129 - val_loss: 0.2515
Epoch 8/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.7715 - loss: 0.0714 - val_acc: 0.5487 - val_loss: 0.2767
Epoch 9/20
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - acc: 0.7806

In [110]:
y_test = model.predict(test_features[0])

57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [111]:
y_test_pred_new = []



for y in y_test:

    lb = []

    for i in range(0, len(y)):

        if y[i] >= 0.5:

            lb.append(1)

        else:

            lb.append(0)

    y_test_pred_new.append(lb)

#Aspect
new_pred = []

new_y = []

for k in range(len(y_test_pred_new)):
    result = np.sum(np.array(y_test_pred_new[k]).reshape(-1,3),axis = 1)
    new_pred.append(result)


for k in range(len(test_features[1])):
    result = np.sum(test_features[1][k].reshape(-1,3),axis = 1)
    new_y.append(result)

In [112]:
print("Aspect + Polarity\n")

print("F1 = {}".format(f1_score(test_features[1], y_test_pred_new) * 100))

print("Precision = {}".format(precision(test_features[1], y_test_pred_new)* 100))

print("Recall = {}".format(recall(test_features[1], y_test_pred_new)* 100))

Aspect + Polarity

F1 = 38.06481481481468
Precision = 46.33333333333333
Recall = 34.12962962962963


In [113]:
print("Aspect\n")

print("F1 = {}".format(f1_score(new_y, new_pred) * 100))

print("Precision = {}".format(precision(new_y, new_pred) * 100))

print("Recall = {}".format(recall(new_y, new_pred) * 100))

Aspect

F1 = 59.70925925925891
Precision = 72.44444444444444
Recall = 53.65740740740744
